In [1]:
import os
import requests
import json
import datetime
import shutil
from bs4 import BeautifulSoup
import pandas as pd
from random import choice
from selenium.common.exceptions import TimeoutException
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse
from time import sleep, time
from random import uniform, randint
import json
from urllib.parse import urlsplit
import urllib3
from glob import glob 

In [2]:
hospital_urls = r'C:\Users\remot\OneDrive\Desktop\Personal\nc-hospital-transparency\data\urls\hospital_data_urls.json'

In [3]:
urls_json = json.load(open(hospital_urls))

In [4]:
print(urls_json.keys())

dict_keys(['duke-university-hospital', 'north-carolina-baptist-hospital', 'university-of-north-carolina-hospital', 'app-regional-health-system', 'atrium-health', 'catawba-valley-medical-center', 'cateret-health-care', 'cone-health', 'first-health-moore', 'first-health-montgomery', 'iredell-health', 'mission-health', 'nhrmc-health', 'northern-regional', 'novant-health', 'vidant-health', 'wakemed-raleigh', 'wakemed-cary'])


In [5]:
duke = urls_json['duke-university-hospital']
north_carolina_bapitist = urls_json['north-carolina-baptist-hospital']
university = urls_json['university-of-north-carolina-hospital']
app = urls_json['app-regional-health-system']
atrium = urls_json['atrium-health']
catawba = urls_json['catawba-valley-medical-center']
cateret = urls_json['cateret-health-care']
cone = urls_json['cone-health']
first = urls_json['first-health-moore']
first = urls_json['first-health-montgomery']
iredell = urls_json['iredell-health']
mission = urls_json['mission-health']
nhrmc = urls_json['nhrmc-health']
northern = urls_json['northern-regional']
novant = urls_json['novant-health']
vidant = urls_json['vidant-health']
wakemed = urls_json['wakemed-raleigh']
wakemed = urls_json['wakemed-cary']

In [6]:
vidant

['https://www.vidanthealth.com/wp-content/uploads/2021/06/56-0989789-VIDANT-NORTH-STANDARD-CHARGES-1.xlsx',
 'https://www.vidanthealth.com/wp-content/uploads/2021/06/56-6011594-VIDANT-DUPLIN-STANDARD-CHARGES.xlsx',
 'https://www.vidanthealth.com/wp-content/uploads/2021/06/56-2112733-THE-OUTER-BANKS-STANDARD-CHARGES.xlsx',
 'https://www.vidanthealth.com/wp-content/uploads/2021/06/56-0585243-VIDANT-MEDICAL-CENTER-STANDARD-CHARGES.xlsx',
 'https://www.vidanthealth.com/wp-content/uploads/2021/06/45-2436270-VIDANT-BEAUFORT-STANDARD-CHARGES.xlsx',
 'https://www.vidanthealth.com/wp-content/uploads/2021/06/56-2072002-VIDANT-BERTIE-STANDARD-CHARGES.xlsx',
 'https://www.vidanthealth.com/wp-content/uploads/2021/06/56-2003393-VIDANT-ROANOKE-CHOWAN-STANDARD-CHARGES.xlsx',
 'https://www.vidanthealth.com/wp-content/uploads/2021/06/56-2093700-VIDANT-EDGECOMBE-STANDARD-CHARGES.xlsx',
 'https://www.vidanthealth.com/wp-content/uploads/2021/06/56-2101090-VIDANT-CHOWAN-STANDARD-CHARGES.xlsx']

In [7]:
# write a function to itarate through the urls and download the csv data with pandas
from modules.scraper import * 

abspath = os.path.normpath(os.path.abspath(''))

raw_download_path = os.path.normpath(os.path.join(abspath, 'data', 'raw'))

url_download_path = os.path.normpath(os.path.join(abspath, 'data', 'urls'))

data_urls_path = os.path.join(url_download_path, 'hospital_data_urls.json')

driver_path = os.path.join(abspath, 'drivers', 'chromedriver.exe')

            

def get_data(url_list, download_path, wakemed=False):
    driver = create_driver(download_path, driver_path)
    for url in url_list:
        if wakemed == True:
            get_wakemed_data(driver, url)
        else:
            get_url_data(driver, url, is_download=True, wait=True)
    


def get_duke(url_list, download_path):
    get_data(url_list, download_path)

def get_north_carolina_bapitist(url_list, download_path):
    get_data(url_list, download_path)

def get_university(url_list, download_path):
    get_data(url_list, download_path)

def get_app(url_list, download_path):
    get_data(url_list, download_path)

def get_atrium(url_list, download_path):
    get_data(url_list, raw_download_path)

def get_catawba(url_list, download_path):
    get_data(url_list, download_path)

def get_cateret(url_list, download_path):
    get_data(url_list, download_path)

def get_cone(url_list, download_path):
    get_data(url_list, download_path)

def get_first(url_list, download_path):
    get_data(url_list, download_path)

def get_iredell(url_list, download_path):
    get_data(url_list, download_path)

def get_mission(url_list, download_path):
    get_data(url_list, download_path)

def get_nhrmc(url_list, download_path):
    get_data(url_list, download_path)

def get_northern(url_list, download_path):
    get_data(url_list, download_path)

def get_novant(url_list, download_path):
    get_data(url_list, download_path)

def get_vidant(url_list, download_path):
    get_data(url_list, download_path)

def get_wakemed(url_list, download_path):
    get_wakemed_data(url_list, download_path)



for file in os.listdir(raw_download_path):
    file_path = os.path.join(raw_download_path, file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)

In [11]:
# get_duke(duke, raw_download_path)
# get_vidant(vidant, raw_download_path)
# get_north_carolina_bapitist(north_carolina_bapitist, raw_download_path)
get_app(app, raw_download_path)
# get_catawba(catawba, raw_download_path)
# get_cateret(cateret, raw_download_path)
# get_cone(cone, raw_download_path)
# get_first(first, raw_download_path)
# get_iredell(iredell, raw_download_path)
# get_mission(mission, raw_download_path)
# get_nhrmc(nhrmc, raw_download_path)
# get_northern(northern, raw_download_path)
# get_novant(novant, raw_download_path)
# get_wakemed(wakemed, raw_download_path)


In [10]:

# , agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36')


In [11]:
os.system("wget -P " + raw_download_path + " " + "https://www.duke.edu/sites/default/files/documents/health/health-data-downloads/2020-03-01-hospital-data-download.csv")

1

In [12]:
os.system("wget --help")

1

In [14]:
import re 
import requests 


def get_filename_from_cd(cd):
    """
    Get filename from content-disposition
    """
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0]


url = 'https://www.duke.edu/sites/default/files/documents/health/health-data-downloads/2020-03-01-hospital-data-download.csv'
r = requests.get(url, allow_redirects=True)
filename = get_filename_from_cd(r.headers.get('content-disposition'))
open(filename, 'wb').write(r.content)

TypeError: expected str, bytes or os.PathLike object, not NoneType